In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
abbr = 'WA'

In [4]:
file = 'washington2018.csv'

Read in federal level data

In [5]:
fiscal = pd.read_sas('../../data/fiscal2018', format = 'sas7bdat', encoding='iso-8859-1')

Generate list of districts in the state in the federal data

In [6]:
state_fiscal = fiscal[(fiscal['STABBR'] == abbr) & (fiscal['GSHI'] == '12')]

In [7]:
len(state_fiscal)

257

In [9]:
state_fiscal.head()

,LEAID,CENSUSID,FIPST,CONUM,CSA,CBSA,NAME,STNAME,STABBR,SCHLEV,...,FL_66V,FL_W01,FL_W31,FL_W61,FL_V95,FL_V02,FL_K14,FL_CE1,FL_CE2,FL_CE3
17787,5300001,48501700600000,53,53033,500,42660,Enumclaw School District,Washington,WA,03,...,R,R,R,R,M,M,M,R,R,M
17788,5300002,48501000300000,53,53019,N,N,Inchelium School District,Washington,WA,03,...,R,R,R,R,M,M,M,R,R,M
17789,5300003,48500800500000,53,53015,440,31020,Kelso School District,Washington,WA,03,...,R,R,R,R,M,M,M,R,R,M
17791,5300005,48500620100000,53,53011,440,38900,Educational Service District 112,Washington,WA,07,...,R,R,R,R,M,M,M,R,R,M
17793,5300007,48501820100000,53,53035,500,14740,Olympic Educational Service District 114,Washington,WA,07,...,R,R,R,R,M,M,M,R,R,M


Read in state level data

In [10]:
state_grads = pd.read_csv('../../data/state_data_raw/' + file)

In [11]:
state_grads.head()

,SchoolYear,OrganizationLevel,County,ESDName,ESDOrganizationId,DistrictCode,DistrictName,DistrictOrganizationId,SchoolCode,SchoolName,...,Year5Dropout,Year6Dropout,Year7Dropout,TransferOut,FinalCohort,Graduate,Continuing,Dropout,GraduationRate,DataAsOf
0,2018,County,Stevens,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/14/2020 12:00:00 AM
1,2018,County,Wahkiakum,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/14/2020 12:00:00 AM
2,2018,County,Walla Walla,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/14/2020 12:00:00 AM
3,2018,County,Whitman,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/14/2020 12:00:00 AM
4,2018,County,Columbia,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,NaN,12/14/2020 12:00:00 AM


Reset columns.

Filter results.

In [12]:
state_grads = state_grads[(state_grads['OrganizationLevel'] == 'District') &
                         (state_grads['StudentGroup'] == 'All Students') &
                         (state_grads['Cohort'] == 'Four Year')]

Select and rename columns.

In [15]:
state_grads = state_grads[['DistrictName', 
                           'FinalCohort',
                           'GraduationRate']]

In [16]:
state_grads.columns = ['District Name', 'Total', 'Graduation Rate']

In [17]:
state_grads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254 entries, 45827 to 77766
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   District Name    254 non-null    object 
 1   Total            201 non-null    float64
 2   Graduation Rate  200 non-null    float64
dtypes: float64(2), object(1)
memory usage: 7.9+ KB


Convert data types.

In [17]:
# state_grads['Total'] = state_grads['Total'].astype(str).str.replace('<', '')
# state_grads['Graduation Rate'] = state_grads['Graduation Rate'].astype(str).str.replace('<', '')

In [18]:
# state_grads['Total'] = pd.to_numeric(state_grads['Total'])
# state_grads['Graduation Rate'] = pd.to_numeric(state_grads['Graduation Rate']) / 100

Check for matches and non-matches in the two lists. 

In [19]:
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.upper().str.strip()
state_grads['District Name'] = state_grads['District Name'].astype(str).str.upper().str.strip()

In [20]:
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(r'\sSu$', '')
# state_grads['District Name'] = state_grads['District Name'].astype(str).str.replace(r'\sSd$', '')

state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' CO PBLC SCHS', ' COUNTY')
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' CITY PBLC SCHS', ' CITY')
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' CTY PBLC SCHS', ' CITY')
state_fiscal['NAME'] = state_fiscal['NAME'].astype(str).str.replace(' PBLC SCHS', ' CITY')

In [22]:
matches = [name for name in list(state_grads['District Name']) if name in list(state_fiscal['NAME'])]
matches.sort()
len(matches)

240

In [20]:
A = [name for name in list(state_grads['District Name']) if name not in list(state_fiscal['NAME'])]
A.sort()
A

['Bellevue School District',
 'Central Valley School District',
 'Clarkston School District',
 'ESD 101 acting as a school district',
 'Highline School District',
 'Lummi Tribal Agency',
 'Marysville School District',
 'Moses Lake School District',
 'Muckleshoot Indian Tribe',
 'North Beach School District No. 64',
 'Orchard Prairie School District',
 'Quileute Tribal School District',
 'Seattle School District No. 1',
 'Wellpinit School District #49']

In [21]:
B = [name for name in list(state_fiscal['NAME']) if name not in list(state_grads['District Name'])]
B.sort()
B

['Centralia College',
 'Clover Park Technical College',
 'ESA 112',
 'Educational Service District 101',
 'Educational Service District 112',
 'Educational Service District 123',
 'Lake Washington Institute of Technology',
 'North Beach School District',
 'Northwest Educational Service District 189',
 'Olympic Educational Service District 114',
 'PRIDE Prep Charter School District',
 'Seattle Public Schools',
 'Valley School District',
 'WA State Center for Childhood Deafness and Hearing Loss',
 'Washington Military Department',
 'Wellpinit School District',
 'Wilson Creek School District']

Make any additional matches I can find.

In [23]:
state_fiscal_rename = {
    'Educational Service District 101' : 'ESD 101 acting as a school district',
    'North Beach School District' : 'North Beach School District No. 64',
    'Seattle Public Schools' : 'Seattle School District No. 1',
    'Wellpinit School District' : 'Wellpinit School District #49'
}

In [24]:
state_fiscal = state_fiscal.replace(state_fiscal_rename)

Merge federal and state data, keeping only matches between the two. 

In [25]:
state_grads_merged = pd.merge(state_fiscal, state_grads, how='inner', left_on='NAME', right_on='District Name')

In [ ]:
state_grads_merged

Save cleaned data. 

In [27]:
state_grads_merged.to_csv('../../data/state_data_merged/' + abbr + '.csv', index=False)